Here we assume that:
* 20 new projects are created on each time step
* each project is assigned a random start time according to *timing_flexibility_vector*
* all projects run for 5 time steps

### TODO:
- currently projects are active before they start, so pending projects are included in the count
- introduce random team size
- produce simple timeseries plots

In [24]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
number_new_projects = 20
timing_flexibility_vector = [0.5,0.25,0.1,0.1,0.05]
project_length = 5
total_timesteps = 1000

In [27]:
def choose_start_time():
    
    x = np.random.uniform(0.0, 1.0)
    
    if x < 0.05:
        return 4
    elif x < 0.15:
        return 3
    elif x < 0.25:
        return 2
    elif x < 0.5:
        return 1
    else:
        return 0

In [28]:
class Project:
    
    def __init__(self, i, start, length=project_length):
        
        self.i = i
        self.start = start
        self.length = length
        self.is_active = True
        self.end = None
        
    def run(self):
        
        if self.is_active:
            
            self.length -= 1
            if self.length == 0:
                self.is_active = False 

In [29]:
project_counter = 0
all_projects = []

for t in range(total_timesteps):
    for p in range(number_new_projects):
        
        all_projects.append(
            Project(project_counter,
                    t + choose_start_time(),
                    project_length))
        project_counter += 1
    
    active_count = sum([p.is_active for p in all_projects])
    print(active_count)
    
    for p in all_projects:
        if t >= p.start:
            p.run()
    
    

20
40
60
80
100
108
112
117
122
119
115
116
120
122
120
119
121
116
117
111
112
119
121
120
119
119
122
125
119
120
124
124
123
124
128
125
112
113
115
116
114
116
114
114
120
114
119
119
120
119
119
114
117
111
108
113
116
116
113
113
120
120
119
117
122
125
121
123
125
123
125
121
121
117
121
125
113
116
116
121
119
120
122
124
121
115
116
113
116
121
122
122
123
116
119
127
122
120
117
118
116
119
120
121
123
122
127
116
115
118
119
121
123
120
119
118
111
115
115
118
123
120
120
117
124
124
123
118
118
124
120
117
121
121
118
120
120
121
126
126
116
119
121
121
120
125
125
122
126
123
121
115
118
120
119
115
114
112
114
114
119
120
117
120
123
123
119
116
122
118
119
114
116
116
115
120
119
119
121
122
122
121
119
120
120
124
121
116
116
124
123
122
115
117
118
121
122
121
118
117
117
113
120
122
122
121
121
123
116
107
111
115
118
111
115
114
121
119
121
115
117
115
117
115
117
118
116
116
121
120
113
114
117
121
119
121
116
125
121
115
119
121
118
122
116
116
114
124
123
122
119
